In [1]:
import sys
from tqdm import tqdm
import pdb
import torch
import logging
import numpy as np
import dgl
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import coo_matrix, csr_matrix

In [2]:
data='TaoBao'
train_path = './datasets/' + data + '/train.txt'
val_path = './datasets/' + data + '/val.txt'
test_path = './datasets/' + data + '/test.txt'
category_path = './datasets/' + data + '/item_category.txt'



In [3]:
def read_category(path):
        num = 0
        dic = {}
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in tqdm(lines):
                line = line.strip().split(',')
                item = int(line[0])
                category = int(line[1])
                if category not in dic.values():
                    num += 1
                dic[item] = category
        return dic, num
category_dic, category_num = read_category(category_path)


100%|██████████| 136710/136710 [00:07<00:00, 18748.27it/s]


In [4]:
def read_train_graph(path):
        historical_dict = {}
        train_data = []
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in tqdm(lines):
                line = line.strip().split(',')
                user = int(line[0])
                item = int(line[1])
                if user in historical_dict:
                    historical_dict[user].add(category_dic[item])
                else:
                    historical_dict[user] = set([category_dic[item]])
        return historical_dict
train=read_train_graph(train_path)
valid=read_train_graph(val_path)
test=read_train_graph(test_path)

100%|██████████| 813098/813098 [00:00<00:00, 1047194.08it/s]


In [5]:
train[0]

{0, 1, 4, 5, 7, 8, 11, 12}

In [6]:
cout=[]
for i in train.keys():
    set1=set(list(train[i]))
    set2=set(list(valid[i]))
    common_elements = set1.intersection(set2)
    #print(len(common_elements))
    cout.append(len(common_elements)/len(set2))
print(sum(cout)/len(cout))

    

0.38345991160646276


In [26]:
cout

[0.0,
 0.25,
 0.6666666666666666,
 0.4,
 0.16666666666666666,
 0.0,
 0.0,
 0.5,
 0.0,
 0.16666666666666666,
 0.5,
 0.3333333333333333,
 0.6666666666666666,
 0.0,
 0.14285714285714285,
 1.0,
 0.2,
 0.0,
 0.42857142857142855,
 0.5,
 0.25,
 0.3333333333333333,
 0.3333333333333333,
 0.18181818181818182,
 0.0,
 0.6666666666666666,
 0.4,
 0.38461538461538464,
 0.5,
 0.5,
 0.0,
 0.3333333333333333,
 0.25,
 0.375,
 0.3333333333333333,
 0.5555555555555556,
 0.0,
 1.0,
 0.8571428571428571,
 0.25,
 0.3333333333333333,
 0.4444444444444444,
 0.3333333333333333,
 0.0,
 0.0,
 0.125,
 0.6,
 0.25,
 0.6,
 0.5,
 0.25,
 0.3333333333333333,
 0.17647058823529413,
 0.42857142857142855,
 0.5714285714285714,
 0.2857142857142857,
 0.3333333333333333,
 0.5,
 0.8,
 0.625,
 0.14285714285714285,
 0.14285714285714285,
 0.6,
 0.25,
 0.0,
 0.125,
 0.125,
 0.6666666666666666,
 0.2857142857142857,
 0.1111111111111111,
 0.6,
 0.0,
 0.25,
 0.4,
 0.0,
 0.0,
 0.0,
 0.3333333333333333,
 0.625,
 0.5,
 0.0,
 0.2,
 0.1111111111

In [34]:
def read_train_graph(path):
        historical_dict = []
        train_data = []
        with open(path, 'r') as f:
            lines = f.readlines()
            for line in tqdm(lines):
                line = line.strip().split(',')
                user = int(line[0])
                item = int(line[1])
                train_data.append([user, item])

                if user in historical_dict:
                    historical_dict[user].add(item)
                else:
                    historical_dict[user] = set([item])

        train_data = torch.tensor(train_data)
        user_number = max(user_number, train_data[:, 0].max() + 1)
        item_number = max(item_number, train_data[:, 1].max() + 1)

        graph_data = {
            ('user', 'rate', 'item'): (train_data[:, 0].long(), train_data[:, 1].long()),
            ('item', 'rated by', 'user'): (train_data[:, 1].long(), train_data[:, 0].long())
        }
        graph = dgl.heterograph(graph_data)
        category_tensor = torch.tensor(list(category_dic.values()), dtype = torch.long).unsqueeze(1)
        graph.ndata['category'] = {'item': category_tensor, 'user': torch.zeros(user_number, 1) - 1}

        return graph

In [8]:
historical_dict = {}
train_data = []
with open(train_path, 'r') as f:
    lines = f.readlines()
    for line in tqdm(lines):
            line = line.strip().split(',')
            user = int(line[0])
            item = int(line[1])
            train_data.append([user, item])

            if user in historical_dict:
                historical_dict[user].append(category_dic[item])
            else:
                historical_dict[user] =[category_dic[item]]
    


100%|██████████| 2571752/2571752 [00:04<00:00, 570483.51it/s]


In [9]:
import math
import random
user_select=[]
maxlen=20
cout=0
for line in tqdm(historical_dict.values()):
    select=[]
    unique_elements = list(set(line))
    element_counts = [line.count(element) for element in unique_elements]
    element_indices = {}
    for index, element in enumerate(line):
        if element in element_indices:
            element_indices[element].append(index)
        else:
            element_indices[element] = [index]
    avg=math.ceil(sum(element_counts)/len(element_counts))
    
    sorted_indices = sorted(range(len(element_counts)), key=lambda i: element_counts[i], reverse=True)
    
    for i in sorted_indices:
        my_list=element_indices[unique_elements[i]]
        if int(len(my_list)/2)>avg:
            
            random_elements = random.sample(my_list,int(len(my_list)/2))
        else:
            random_elements=random.sample(my_list, k=min(avg,len(my_list)))
        select=select+random_elements
    if len(select)>=maxlen:
        select=select[0:maxlen]
    else:
        select=select+select[0:maxlen-len(select)]
        cout=cout+1
    user_select.append(select)
    break
    


  0%|          | 0/82633 [00:00<?, ?it/s]


In [94]:
cout

43762

In [12]:
print(line)
print(unique_elements)
print(element_counts)
print(sorted_indices)
print(element_indices[unique_elements[3]])

[0, 1, 4, 5, 4, 5, 1, 7, 5, 8, 11, 5, 5, 5, 5, 12]
[0, 1, 4, 5, 7, 8, 11, 12]
[1, 2, 2, 7, 1, 1, 1, 1]
[3, 1, 2, 0, 4, 5, 6, 7]
[3, 5, 8, 11, 12, 13, 14]


In [71]:
print(element_indices)

{0: [0], 1: [1, 6], 4: [2, 4], 5: [3, 5, 8, 11, 12, 13, 14], 7: [7], 8: [9], 11: [10], 12: [15]}


In [72]:
print(user_select)

[[3, 8, 1, 6, 2, 4, 0]]


In [79]:
a=[1,2,3,4]
print(a[-1:])

[4]
